<a href="https://colab.research.google.com/github/zmgy107/Pytorch-Learning-Notes/blob/main/Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Optimizing Model Parameters

Now that we have a model and data it’s time to train, validate and test our model by optimizing its parameters on our data. Training a model is an iterative process; in each iteration the model makes a guess about the output, calculates the error in its guess (loss), collects the derivatives of the error with respect to its parameters (as we saw in the previous section), and __optimizes__ these parameters using gradient descent. 

# Prerequisite Code

Load tha code from the previous section

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader=DataLoader(training_data,batch_size=64)
test_dataloader=DataLoader(test_data,batch_size=64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork,self).__init__()
    self.flatten=nn.Flatten()
    self.linear_relu_stack=nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
    )
  def forward(self,x):
    x=self.flatten(x)
    logits=self.linear_relu_stack(x)
    return logits

model=NeuralNetwork()

# Hyperparameters

Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates

*   __Number of Epochs__:the number times to iterate over the dataset(the number of dataset/Batch Size)
*   __Batch Size__:the number of data samples propagated through the network before the parameters are updated
*   __Learning Rate__:how much to update models parameters at each batch/epoch.Smaller values yield slow learning speed,while large values may result in unpredictable behavior during training.

In [14]:
learning_rate=1e-3
batch_size=64
epochs=5

# Optimization Loop

Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an epoch.

Each epoch consists of two main parts:

*   __The Train Loop__:iterate over the training dataset and try to converge to optimal parameters.
*   __The Validation/Test Loop__:iterate over the test dataset to check if model performance is improving.



# Loss Function

When presented with some training data, our untrained network is likely not to give the correct answer. Loss function measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include nn.MSELoss (Mean Square Error) for regression tasks, and nn.NLLLoss (Negative Log Likelihood) for classification. nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss.

We pass our model’s output logits to nn.CrossEntropyLoss, which will normalize the logits and compute the prediction error.

In [15]:
# Initialize the loss function
loss_fn=nn.CrossEntropyLoss()

# Optimizer

Optimization is the process of adjusting model parameters to reduce model error in each training step. __Optimization algorithms__ define how this process is performed (in this example we use Stochastic Gradient Descent). All optimization logic is encapsulated in the optimizer object. Here, we use the SGD optimizer; additionally, there are many different optimizers available in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data.

We initialize the optimizer by registering the model’s parameters that need to be trained, and passing in the learning rate hyperparameter.

In [16]:
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

Inside the training loop,optimization happens in steps:

*   Call optimizer.zero_grad() to reset the gradients of model parameters.Gradients by default add up;to prevent double-counting,we explicitly zero them at each iteration.
*   Backpropagate the prediction loss with a call to loss.backward().Pytorch deposits the gradients of the loss w.r.t. each parameter.
*   Once we have our gradients,we call optimizer.step() to adjust the parameters by the gradients collected in the backward pass.


# Full Implementation

We define train_loop that loops over our optimization code, and test_loop that evaluates the model’s performance against our test data.

In [17]:
def train_loop(dataloader,model,loss_fn,optimizer):
  size=len(dataloader.dataset)
  for batch,(X,y) in enumerate(dataloader):
    # Compute prediction and loss
    pred=model(X)
    loss=loss_fn(pred,y)

    #Backpropopation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100==0:
      loss,current=loss.item(),(batch+1)*len(X)
      print(f"loss:{loss:>7f}[{current:>5d}/{size:>5d}]")

def test_loop(dataloader,model,loss_fn):
  size=len(dataloader.dataset)
  num_batches=len(dataloader)
  test_loss,correct=0,0

  with torch.no_grad():
    for X,y in dataloader:
      pred=model(X)
      test_loss+=loss_fn(pred,y).item()
      correct+=(pred.argmax(1)==y).type(torch.float).sum().item()

  test_loss/=num_batches
  correct/=size
  print(f"Test Error:\n Accuracy:{(100*correct):>0.1f}%,Avg loss:{test_loss:>8f}\n")

We initialize the loss function and optimizer, and pass it to train_loop and test_loop. Feel free to increase the number of epochs to track the model’s improving performance.

In [18]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

epochs=20
for t in range(epochs):
  print(f"Epoch{t+1}\n-------------------------------")
  train_loop(train_dataloader,model,loss_fn,optimizer)
  test_loop(test_dataloader,model,loss_fn)
print("Done!")

Epoch1
-------------------------------
loss:2.290461[   64/60000]
loss:2.286208[ 6464/60000]
loss:2.260115[12864/60000]
loss:2.263391[19264/60000]
loss:2.248230[25664/60000]
loss:2.196564[32064/60000]
loss:2.217726[38464/60000]
loss:2.170614[44864/60000]
loss:2.158456[51264/60000]
loss:2.132320[57664/60000]
Test Error:
 Accuracy:38.8%,Avg loss:2.132045

Epoch2
-------------------------------
loss:2.138399[   64/60000]
loss:2.131907[ 6464/60000]
loss:2.068871[12864/60000]
loss:2.095905[19264/60000]
loss:2.042340[25664/60000]
loss:1.963003[32064/60000]
loss:1.999209[38464/60000]
loss:1.909289[44864/60000]
loss:1.907547[51264/60000]
loss:1.839177[57664/60000]
Test Error:
 Accuracy:58.1%,Avg loss:1.846909

Epoch3
-------------------------------
loss:1.875422[   64/60000]
loss:1.850544[ 6464/60000]
loss:1.730170[12864/60000]
loss:1.784598[19264/60000]
loss:1.674016[25664/60000]
loss:1.614184[32064/60000]
loss:1.635826[38464/60000]
loss:1.538181[44864/60000]
loss:1.563007[51264/60000]
loss:1